In [2]:
def load_to_db():
    cutoff = input("Nhập kỳ CUTOFF: ")
    excel_file = '{}.xlsx'.format(cutoff)
    import pandas as pd
    import pyodbc as pyo
    handle_excel = pd.ExcelFile(excel_file)
    agent = handle_excel.parse('WCC_Agent')
    policy = handle_excel.parse('WCC_Policy')
    agent_call = agent
    policy_call = policy
    #3.Tạo Connection
    conn_toSQL = pyo.connect(driver='{ODBC Driver 17 for SQL Server}', host = 'TEDPHO0134\TANDT_DP', database = 'DP_Tan', trusted_connection = 'yes',
                        ansi=True,autocommit=True)
    cursor = conn_toSQL.cursor()
    #Đổi tên cột
    agent_call.rename(columns = {'Period Cập nhật ':"updatedPeriod",
     'Period Gỡ':"duePeriod",
     'WCC ROOKIES / WCC TRAINEE':"rookieTrainee",
     'Tháng gia nhập':"appointedDate",
     'Ngày thực hiện cuộc gọi gần nhất ':'recentCallDate',
     'KẾT QUẢ WCC':"Result"}, inplace = True) 
    policy_call.rename(columns={
    'Period Cập nhật ': "updatedPeriod",
    'Period Gỡ': "duePeriod",
     'SALES UNIT': "SalesUnit",
     'DATE OF LATEST WCC': "LastedWCC",
     'FINAL WCC RESULT':"finalResult"
    }, inplace = True)
    #Tạo cột CUTOFF
    agent_call['CUTOFF'] = cutoff
    policy_call['CUTOFF'] = cutoff
    #Đẩy vào Database:
    agent_call = agent_call.fillna(value=0) #fillna để tránh lỗi trong việc đẩy data vào SQL
    #Đẩy dữ liệu vào Bảng WCC_Agent
    for index, row in agent_call.iterrows():
         cursor.execute("INSERT INTO dbo.WCC_Agent (updatedPeriod,duePeriod,Agent_Number,Agent_Name,rookieTrainee,appointedDate,recentCallDate,Result,CUTOFF) values(?,?,?,?,?,?,?,?,?)"
        , 
        row.updatedPeriod,
        row.duePeriod,
        row.Agent_Number,
        row.Agent_Name,
        row.rookieTrainee,
        row.appointedDate,
        row.recentCallDate,
        row.Result,
        row.CUTOFF
    )
    conn_toSQL.commit()
    policy_call = policy_call.fillna(value=0)
    #Đẩy dữ liệu vào Bảng WCC_Policy
    for index, row in policy_call.iterrows():
         cursor.execute("INSERT INTO dbo.WCC_Policy (updatedPeriod,duePeriod,CONTRACT_NUMBER,PO_NAME,POLICY_STATUS,AGENT_MOBILE,SALE_OFFICE,AGENT_CODE,AGENT_NAME,AGENT_GRADE,ISSUE_DATE,SalesUnit,LastedWCC,finalResult,CUTOFF) values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"
    , 
    row.updatedPeriod,
    row.duePeriod,
    row.CONTRACT_NUMBER,
    row.PO_NAME,
    row.POLICY_STATUS,
    row.AGENT_MOBILE,
    row.SALE_OFFICE,
    row.AGENT_CODE,
    row.AGENT_NAME,
    row.AGENT_GRADE,
    row.ISSUE_DATE,
    row.SalesUnit,
    row.LastedWCC,
    row.finalResult,
    row.CUTOFF
    )
    conn_toSQL.commit()
    return "Data loaded Successfully!"

In [3]:
load_to_db()

Nhập kỳ CUTOFF: 20240229


'Data loaded Successfully!'